# Ornstein-Uhlenbeck Process Simulation

The Ornstein-Uhlenbeck SDE models a mean reverting random process and is defined by,

$
\begin{align}
dX_t = \lambda \left( \mu - X_t \right) dt + \sigma dB_t
\end{align}
$

where $dB_t \sim \text{Normal}(0, dt)$.

The model can be descritized using the Euler method from continuous differential equations. The stochastic version of it is called</br>
the [Euler-Maruyama](https://en.wikipedia.org/wiki/Euler%E2%80%93Maruyama_method) method. Replacing the continuous differential with finite differences gives,

$
\begin{align}
&\Delta X_t = \lambda \left( \mu - X_t \right) \Delta t + \sigma \Delta B_t \\
\Rightarrow &X_{t+1} = X_t + \lambda \left( \mu - X_t \right) \Delta t + \sigma \Delta B_t
\end{align}
$

but $\Delta B_t \sim \text{Normal}(0, \Delta t)$. It follows that the simulated model is given by,

$
\begin{align}
X_{t+1} = X_t + \lambda \left( \mu - X_t \right) \Delta t + \sigma \Delta t\hspace{2pt} \varepsilon_t
\end{align}
$

where $\varepsilon \sim \text{Normal}(0,1)$.

The solution to the Ornstein-Uhlenbeck SDE is given by,

$
\begin{align}
X_t &= \text{E}[X_t] + \sqrt{\text{Var}(X_t)}\hspace{2pt}\varepsilon_t \\
&= X_0 e^{-\lambda t} + \mu \left( 1 - e^{-\lambda t} \right) + \sqrt{ \frac{\sigma^2}{2\lambda} \left( 1 - e^{-2\lambda t} \right)} \hspace{5pt} \varepsilon_t \hspace{30pt} (4)
\end{align}
$

where $\varepsilon_t \sim \text{Normal}(0,1)$.


## Imports